<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 10.8 MB/s eta 0:00:00


In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [21]:
# use requests.get() method with the provided static_url
import requests

static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Sending a GET request to the URL
response = requests.get(static_url)

# Checking if the request was successful (status code 200 indicates success)
if response.status_code == 200:
    print("Request successful!")
else:
    print("Request failed with status code:", response.status_code)



Request successful!


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.text, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute
title = soup.title
if title:
    print("Title:", title.text)
else:
    print("Title not found in the HTML document.")

Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
html_tables = soup.find_all('table')
print("Number of tables found:", len(html_tables))

Number of tables found: 25


Starting from the third table is our target table contains the actual launch records.


In [9]:
# Let's print the third table and check its content
third_table = html_tables[2]
print(third_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [10]:
column_names = [extract_column_from_header(th) for th in third_table.find_all('th')]
print("Column names:", column_names)

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


Column names: ['Flight No.', 'Date and time ( )', '', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', '', None, None, None, None, None, None, None]


Check the extracted column names


In [11]:
print(column_names)

['Flight No.', 'Date and time ( )', '', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', '', None, None, None, None, None, None, None]


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [33]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

df = pd.DataFrame(launch_dict)
print(df.head())

Empty DataFrame
Columns: [Flight No., , Launch site, Payload, Payload mass, Orbit, Customer, Launch outcome, None, Version Booster, Booster landing, Date, Time]
Index: []


In [23]:
 #URL de la API de SpaceX para obtener información sobre lanzamientos
url = "https://api.spacexdata.com/v4/launches"

# Realizar solicitud GET a la API de SpaceX y asignarla a un objeto response
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Convertir la respuesta JSON en un DataFrame utilizando pd.json_normalize
    response_json = response.json()  # Convertir la respuesta JSON en un objeto
    df = pd.json_normalize(response_json)
    
    # Extraer el año de la primera fila de la columna static_fire_date_utc
    first_static_fire_date_utc = df.at[0, 'static_fire_date_utc']
    year = first_static_fire_date_utc.split('-')[0] if first_static_fire_date_utc else "No se encontró fecha de prueba estática"

    print("El año en la primera fila de la columna static_fire_date_utc es:", year)
else:
    print("Error al realizar la solicitud a la API de SpaceX. Código de estado:", response.status_code)


El año en la primera fila de la columna static_fire_date_utc es: 2006


In [47]:
import requests
import pandas as pd

# URL de la API de SpaceX
url = "https://api.spacexdata.com/v5/launches"

# Realizar una solicitud GET a la API
response = requests.get(url)

# Convertir la respuesta JSON a un DataFrame de Pandas
data = pd.DataFrame(response.json())

# Inspeccionar la estructura del DataFrame
print(data.head())

# Intentar acceder al nombre del cohete para entender su estructura
rocket_name = data["rocket"].apply(lambda x: x.get("name"))
print(rocket_name)


                                            fairings  \
0  {'reused': False, 'recovery_attempt': False, '...   
1  {'reused': False, 'recovery_attempt': False, '...   
2  {'reused': False, 'recovery_attempt': False, '...   
3  {'reused': False, 'recovery_attempt': False, '...   
4  {'reused': False, 'recovery_attempt': False, '...   

                                               links  \
0  {'patch': {'small': 'https://images2.imgbox.co...   
1  {'patch': {'small': 'https://images2.imgbox.co...   
2  {'patch': {'small': 'https://images2.imgbox.co...   
3  {'patch': {'small': 'https://images2.imgbox.co...   
4  {'patch': {'small': 'https://images2.imgbox.co...   

       static_fire_date_utc  static_fire_date_unix    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                      None                    NaN  False     0.0   
2                      None                    NaN  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09

AttributeError: 'str' object has no attribute 'get'

In [49]:

import requests
import pandas as pd

# URL de la API de SpaceX
url = "https://api.spacexdata.com/v5/launches"

# Realizar una solicitud GET a la API
response = requests.get(url)

# Convertir la respuesta JSON a un DataFrame de Pandas
data = pd.DataFrame(response.json())

# Contar los valores faltantes en la columna 'landingPad'
missing_landing_pad = data['landpad'].isnull().sum()

print("Número de valores faltantes en la columna 'landingPad':", missing_landing_pad)



    

KeyError: 'landpad'

In [54]:
import requests
import pandas as pd

# URL de la API de SpaceX
url = "https://api.spacexdata.com/v5/launches"

# Realizar una solicitud GET a la API
response = requests.get(url)

# Convertir la respuesta JSON a un DataFrame de Pandas
data = pd.json_normalize(response.json())

# Verificar las primeras filas del DataFrame para entender la estructura
print(data)


         static_fire_date_utc  static_fire_date_unix    net  window  \
0    2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                        None                    NaN  False     0.0   
2                        None                    NaN  False     0.0   
3    2008-09-20T00:00:00.000Z           1.221869e+09  False     0.0   
4                        None                    NaN  False     0.0   
..                        ...                    ...    ...     ...   
200                      None                    NaN  False     NaN   
201                      None                    NaN  False     NaN   
202                      None                    NaN  False     NaN   
203                      None                    NaN  False     NaN   
204                      None                    NaN  False     NaN   

                       rocket success  \
0    5e9d0d95eda69955f709d1eb   False   
1    5e9d0d95eda69955f709d1eb   False   
2    5e9d0d95eda69955f70

In [45]:
import pandas as pd

#Supongamos que `df` es el marco de datos resultante de la solicitud GET a la API de Space X.

#Obtenemos la fecha del primer encendido estático.
static_fire_date = df['static_fire_date_utc'].iloc[0]

#Extraemos el año.
year = static_fire_date.dt.year

#Imprimimos el año.
print(year)

AttributeError: 'str' object has no attribute 'dt'

In [55]:
import requests

# URL de la API de SpaceX para obtener información sobre lanzamientos
url = "https://api.spacexdata.com/v3/launches"

# Realizar solicitud GET a la API de SpaceX y obtener la respuesta
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Convertir la respuesta JSON en un objeto Python
    launches = response.json()
    
    # Filtrar los lanzamientos para incluir solo los de Falcon 9 y contarlos
    falcon_9_launches = [launch for launch in launches if 'Falcon 9' in launch.get('rocket')]
    num_falcon_9_launches = len(falcon_9_launches)
    
    # Filtrar los lanzamientos para excluir los de Falcon 1 y contarlos
    falcon_1_launches = [launch for launch in launches if 'Falcon 1' in launch.get('rocket')]
    num_falcon_1_launches = len(falcon_1_launches)
    
    # Calcular el número de lanzamientos de Falcon 9 después de eliminar los de Falcon 1
    num_falcon_9_after_falcon_1 = num_falcon_9_launches - num_falcon_1_launches
    
    print("Número de lanzamientos de Falcon 9 después de eliminar los de Falcon 1:", num_falcon_9_after_falcon_1)
else:
    print("Error al realizar la solicitud a la API de SpaceX. Código de estado:", response.status_code)


Número de lanzamientos de Falcon 9 después de eliminar los de Falcon 1: 0


In [35]:
import requests

# URL de la API de SpaceX para obtener información sobre lanzamientos
url = "https://api.spacexdata.com/v4/launches"

# Realizar solicitud GET a la API de SpaceX y obtener la respuesta
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Convertir la respuesta JSON en un objeto Python
    launches = response.json()
    
    # Contar el número de lanzamientos con valores faltantes en la columna landingPad
    missing_values_count = sum('landPad' not in launch for launch in launches)
    
    print("Número de valores faltantes para la columna landingPad:", missing_values_count)
else:
    print("Error al realizar la solicitud a la API de SpaceX. Código de estado:", response.status_code)


Número de valores faltantes para la columna landingPad: 205


In [36]:
import requests
import pandas as pd

# URL de la API de SpaceX para obtener información sobre lanzamientos
url = "https://api.spacexdata.com/v4/launches"

# Realizar solicitud GET a la API de SpaceX y obtener la respuesta
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Convertir la respuesta JSON en un DataFrame de pandas
    launches_df = pd.DataFrame(response.json())
    
    # Mostrar las primeras filas del DataFrame para verificar la conversión
    print(launches_df.head())
else:
    print("Error al realizar la solicitud a la API de SpaceX. Código de estado:", response.status_code)


                                            fairings  \
0  {'reused': False, 'recovery_attempt': False, '...   
1  {'reused': False, 'recovery_attempt': False, '...   
2  {'reused': False, 'recovery_attempt': False, '...   
3  {'reused': False, 'recovery_attempt': False, '...   
4  {'reused': False, 'recovery_attempt': False, '...   

                                               links  \
0  {'patch': {'small': 'https://images2.imgbox.co...   
1  {'patch': {'small': 'https://images2.imgbox.co...   
2  {'patch': {'small': 'https://images2.imgbox.co...   
3  {'patch': {'small': 'https://images2.imgbox.co...   
4  {'patch': {'small': 'https://images2.imgbox.co...   

       static_fire_date_utc  static_fire_date_unix    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                      None                    NaN  False     0.0   
2                      None                    NaN  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09

In [44]:
import pandas as pd
import requests


url = 'https://api.spacexdata.com/v5/launches'


response = requests.get(url)

if response.status_code == 200:
    # Convertir la respuesta JSON a un DataFrame
    df = pd.json_normalize(response.json())

    # Contar los valores nulos en la columna 'landingPad'
    null_values = df['landingPad'].isnull().sum()

    # Imprimir el número de valores nulos
    print(f"Número de valores nulos en 'landingPad': {null_values}")
else:
    print("Error al obtener los datos de la API.")


KeyError: 'landingPad'

In [43]:
num_missing_landing_pad = launches_df['launchpad'].isnull().sum()
print("Número de valores faltantes en la columna 'landingPad':", num_missing_landing_pad)


Número de valores faltantes en la columna 'landingPad': 0


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [13]:
extracted_row = 0

# Extract each table
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # get table row
    for rows in table.find_all("tr"):
        # check to see if first table heading is a number corresponding to launch a number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        # get table element
        row = rows.find_all('td')
        # if it is number save cells in a dictionary
        if flag:
            extracted_row += 1
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)

            # Date and Time
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',') if datatimelist[0] else ''
            time = datatimelist[1] if datatimelist[1] else ''
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

            # Booster version
            bv = booster_version(row[1])
            if not bv and row[1].a:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            launch_site = row[2].a.string if row[2].a else ''
            launch_dict['Launch site'].append(launch_site)

            # Payload
            payload = row[3].a.string if row[3].a else ''
            launch_dict['Payload'].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Orbit
            orbit = row[5].a.string if row[5].a else ''
            launch_dict['Orbit'].append(orbit)

            # Customer
            customer = row[6].a.string if row[6].a else ''
            launch_dict['Customer'].append(customer)

            # Launch outcome
            launch_outcome = list(row[7].strings)[0] if row[7].a else ''
            launch_dict['Launch outcome'].append(launch_outcome)

            # Booster landing
            booster_landing = landing_status(row[8]) if row[8].a else ''
            launch_dict['Booster landing'].append(booster_landing)

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [14]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [15]:
df.to_csv('spacex_web_scraped.csv', index=False)

In [19]:
# URL de la API de SpaceX para obtener información sobre lanzamientos
url = "https://api.spacexdata.com/v4/launches"

# Realizar solicitud GET a la API de SpaceX
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:


df = pd.json_normalize(response.json())
first_static_fire_date_utc = df.at[0, 'static_fire_date_utc']
year = first_static_fire_date_utc.split('-')[0] if first_static_fire_date_utc else "No se encontró fecha de prueba estática"

print("El año en la primera fila de la columna static_fire_date_utc es:", year)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
